In [5]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

# Load the updated dataset
data = pd.read_csv('data/part_1/historical_weather_without_outliers.csv')

# Convert the 'date' column to datetime format
data['date'] = pd.to_datetime(data['date'])

# Set the 'date' column as the index
data.set_index('date', inplace=True)

# Function to train and evaluate an ETS model with given parameters
def train_evaluate_ets_model(city_data, seasonal, seasonal_periods):
    # Split the data into training and testing sets
    tscv = TimeSeriesSplit(n_splits=3)
    errors = []

    for train_index, test_index in tscv.split(city_data):
        train, test = city_data.iloc[train_index], city_data.iloc[test_index]

        # Define and fit the ETS model
        model = ExponentialSmoothing(train['avg_temp_c'], seasonal=seasonal, seasonal_periods=seasonal_periods)
        model_fit = model.fit()

        # Forecast the test set
        forecast = model_fit.forecast(len(test))
        error = mean_squared_error(test['avg_temp_c'], forecast)
        errors.append(error)

    return np.mean(errors)

# Hyperparameters to try
seasonal_options = ['add', 'mul']
seasonal_periods_options = [12, 24]

# Dictionary to store optimized ETS models for each city
optimized_ets_models = {}

# Train and optimize an ETS model for each city
for city_id, city_data in data.groupby('city_id'):
    best_error = float('inf')
    best_params = None
    best_model = None

    for seasonal in seasonal_options:
        # Skip multiplicative option if any value in the series is non-positive
        if seasonal == 'mul' and (city_data['avg_temp_c'] <= 0).any():
            continue

        for seasonal_periods in seasonal_periods_options:
            error = train_evaluate_ets_model(city_data, seasonal, seasonal_periods)
            if error < best_error:
                best_error = error
                best_params = (seasonal, seasonal_periods)
                best_model = ExponentialSmoothing(city_data['avg_temp_c'], seasonal=seasonal, seasonal_periods=seasonal_periods).fit()

    optimized_ets_models[city_id] = best_model
    print(f"Optimized ETS model for city {city_id}: Seasonal: {best_params[0]}, Seasonal Periods: {best_params[1]}")

# Function to forecast using the optimized ETS model
def forecast_optimized_ets_model(model, steps):
    forecast = model.forecast(steps=steps)
    return forecast

# Dictionary to store forecasted values for each city
optimized_ets_forecasts = {}

# Forecast the first week of 2019 for each city and print the forecasted values
for city_id, model in optimized_ets_models.items():
    forecast = forecast_optimized_ets_model(model, steps=7)
    optimized_ets_forecasts[city_id] = forecast
    print(f"Forecasted values for city {city_id}:")
    print(forecast)
    print("\n")

# Save the forecasted values to a CSV file
forecast_data = []
submission_id = 1
for city_id, forecast in optimized_ets_forecasts.items():
    for temp in forecast:
        forecast_data.append({'submission_ID': submission_id, 'avg_temp_c': round(temp, 2)})
        submission_id += 1

forecast_df = pd.DataFrame(forecast_data)
forecast_df.to_csv('data/part_1/forcasted_data/ets.csv', index=False)

print("Optimized forecast data has been saved to 'optimized_ets_forecasted_temperatures.csv'")


d:\Competitions\Predicta 1.0 (2024)\Predicta_Solutions\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
d:\Competitions\Predicta 1.0 (2024)\Predicta_Solutions\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
d:\Competitions\Predicta 1.0 (2024)\Predicta_Solutions\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
d:\Competitions\Predicta 1.0 (2024)\Predicta_Solutions\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
d:\Competitions\Predicta 1.0 (2024)\

Optimized ETS model for city C001: Seasonal: add, Seasonal Periods: 24


d:\Competitions\Predicta 1.0 (2024)\Predicta_Solutions\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
d:\Competitions\Predicta 1.0 (2024)\Predicta_Solutions\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
d:\Competitions\Predicta 1.0 (2024)\Predicta_Solutions\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\Competitions\Predicta 1.0 (2024)\Predicta_Solutions\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No

Optimized ETS model for city C002: Seasonal: add, Seasonal Periods: 12


d:\Competitions\Predicta 1.0 (2024)\Predicta_Solutions\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
d:\Competitions\Predicta 1.0 (2024)\Predicta_Solutions\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
d:\Competitions\Predicta 1.0 (2024)\Predicta_Solutions\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\Competitions\Predicta 1.0 (2024)\Predicta_Solutions\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A 

...

Optimized ETS model for city C111: Seasonal: add, Seasonal Periods: 24


d:\Competitions\Predicta 1.0 (2024)\Predicta_Solutions\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
d:\Competitions\Predicta 1.0 (2024)\Predicta_Solutions\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
d:\Competitions\Predicta 1.0 (2024)\Predicta_Solutions\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\Competitions\Predicta 1.0 (2024)\Predicta_Solutions\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No

Optimized ETS model for city C112: Seasonal: mul, Seasonal Periods: 12
Forecasted values for city C001:
2019-01-01    11.720644
2019-01-02    11.428123
2019-01-03    11.235613
2019-01-04    11.368116
2019-01-05    11.254560
2019-01-06    11.290995
2019-01-07    11.453729
Freq: D, dtype: float64


Forecasted values for city C002:
1803    13.812597
1804    13.799367
1805    13.748140
1806    13.842906
1807    13.877012
1808    13.749117
1809    13.876556
dtype: float64


Forecasted values for city C003:
1823    27.131960
1824    27.265034
1825    27.231511
1826    27.158282
1827    27.102402
1828    27.119130
1829    27.121202
dtype: float64


Forecasted values for city C004:
2019-01-01   -0.920041
2019-01-02   -1.314592
2019-01-03   -1.453256
2019-01-04   -1.212934
2019-01-05   -1.035873
2019-01-06   -1.264608
2019-01-07   -1.370984
Freq: D, dtype: float64


Forecasted values for city C005:
2019-01-01    26.049773
2019-01-02    25.701301
2019-01-03    26.019027
2019-01-04    26.607050
2

d:\Competitions\Predicta 1.0 (2024)\Predicta_Solutions\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
d:\Competitions\Predicta 1.0 (2024)\Predicta_Solutions\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
d:\Competitions\Predicta 1.0 (2024)\Predicta_Solutions\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
d:\Competitions\Predicta 1.0 (2024)\Predicta_Solutions\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported inde

Forecasted values for city C112:
1800    20.965222
1801    20.641008
1802    20.644574
1803    21.120602
1804    20.961491
1805    20.704761
1806    21.054947
dtype: float64


Optimized forecast data has been saved to 'optimized_ets_forecasted_temperatures.csv'
